In [27]:
import pandas as pd

In [28]:
csvpath = 'Resources/purchase_data.csv'
purchases_df = pd.read_csv(csvpath)


In [29]:
# Total Players
total_players = len(purchases_df['SN'].unique())
pd.DataFrame({'Total Players':[total_players]})

,Total Players
0,576


In [30]:
# Purchasing Analysis

## Total Items
total_items = len(purchases_df['Item ID'].unique())


# Average Purchase Price
price_sum = purchases_df['Price'].sum()
average_purchase_price = price_sum/total_items


# Total Purchases
total_purchases = len(purchases_df)



summary_df = pd.DataFrame({'Number of Unique Items':[total_items],'Average Purchase Price':[average_purchase_price],
                          'Total Purchases':[total_purchases],'Total Revenue':[price_sum]})
summary_df['Average Purchase Price'] = summary_df['Average Purchase Price'].map('${:,.2f}'.format)
summary_df['Total Revenue'] = summary_df['Total Revenue'].map('${:,.2f}'.format)
summary_df

,Number of Unique Items,Average Purchase Price,Total Purchases,Total Revenue
0,179,$13.29,780,"$2,379.77"


In [31]:
## Gender Demographics

## Create new data representing Users, Gender and Age
user_info_df = purchases_df[['SN','Gender','Age']]
clean_user_df = user_info_df.drop_duplicates(keep='first')

# Find out unique values in gender column
genders = user_info_df['Gender'].unique()

## Percentage and Count of Female Players
female_players = clean_user_df.loc[clean_user_df['Gender']=='Female']
total_female = len(female_players)
percent_female = (total_female/total_players)*100

## Percentage and Count of Male Players
male_players = clean_user_df.loc[clean_user_df['Gender']=='Male']
total_male = len(male_players)
percent_male = (total_male/total_players)*100

## Percentage and Count of Other / Non-Disclosed
other_players = clean_user_df.loc[clean_user_df['Gender']=='Other / Non-Disclosed']
total_other = len(other_players)
percent_other = (total_other/total_players)*100


gender_summary_df = pd.DataFrame({'Gender':genders,'Total Players':[total_male,total_other,total_female],
                                  'Percent of Total Players':[percent_male,percent_other,percent_female]})

gender_summary_df['Percent of Total Players'] = gender_summary_df['Percent of Total Players'].map('{:,.2f}%'.format)
gender_summary_df

,Gender,Total Players,Percent of Total Players
0,Male,484,84.03%
1,Other / Non-Disclosed,11,1.91%
2,Female,81,14.06%


In [32]:
### Purchasing Analysis (Gender) - Purchase Count

## Female total purchases
female_df = purchases_df.loc[purchases_df['Gender']=='Female']
female_count = female_df['Item ID'].count()


## Male total purchases
male_df = purchases_df.loc[purchases_df['Gender']=='Male']
male_count = male_df['Item ID'].count()


## Other/Non-Disclosed total purchases
other_df = purchases_df.loc[purchases_df['Gender']=='Other / Non-Disclosed']
other_count = other_df['Item ID'].count()


gender_group_df = purchases_df.groupby(['Gender']).sum()

## Female 
female_total_price = gender_group_df.iloc[0,3]
female_ave_price = female_total_price/female_count
price_per_female = female_total_price/total_female


## Male
male_total_price = gender_group_df.iloc[1,3]
male_ave_price = male_total_price/male_count
price_per_male = male_total_price/total_male


## Other/Non-disclosed
other_total_price = gender_group_df.iloc[2,3]
other_ave_price = other_total_price/other_count
price_per_other = other_total_price/total_other

gender_purchase_df = pd.DataFrame({'Gender':genders,'Total Purchased':[male_count,other_count,female_count],
                                  'Average Purchase Price':[male_ave_price,other_ave_price,female_ave_price],
                                  'Total Purchase Value':[male_total_price,other_total_price,female_total_price],
                                  'Average Price Per Gender':[price_per_male,price_per_other,price_per_female]})

gender_purchase_df['Average Purchase Price'] = gender_purchase_df['Average Purchase Price'].map('${:,.2f}'.format)
gender_purchase_df['Total Purchase Value'] = gender_purchase_df['Total Purchase Value'].map('${:,.2f}'.format)
gender_purchase_df['Average Price Per Gender'] = gender_purchase_df['Average Price Per Gender'].map('${:,.2f}'.format)


gender_purchase_df

,Gender,Total Purchased,Average Purchase Price,Total Purchase Value,Average Price Per Gender
0,Male,652,$3.02,"$1,967.64",$4.07
1,Other / Non-Disclosed,15,$3.35,$50.19,$4.56
2,Female,113,$3.20,$361.94,$4.47


In [33]:
## Age Demographics - Total and Percentage of players by age group

# create bins to sort users into age groups 
bins = [0,9.90,14.90,19.90,24.90,29.90,34.90,39.90,1000]
age_groups = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

# Group by Data Frame to get count, mean and sum by age group
purchases_df['Age Group'] = pd.cut(purchases_df['Age'], bins, labels = age_groups, include_lowest=True)
age_group_df = purchases_df.groupby(['Age Group'])
age_total = age_group_df['SN'].nunique()

# Calculate Percentages for each age group
binz = []
for j in range(0,8):
    y = (age_total[j]/total_players)*100
    binz.append(y)

# Create new Data Frame for Number and Percentage of Players
age_group_summary_df = pd.DataFrame({'Age Group':['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+'],
                                    'Number of Players':[age_total[0],age_total[1],age_total[2],age_total[3],
                                                        age_total[4],age_total[5],age_total[6],age_total[7]],
                                    'Percentage':binz})

age_group_summary_df['Percentage'] = age_group_summary_df['Percentage'].map('{:,.2f}%'.format)

age_group_summary_df


,Age Group,Number of Players,Percentage
0,<10,17,2.95%
1,10-14,22,3.82%
2,15-19,107,18.58%
3,20-24,258,44.79%
4,25-29,77,13.37%
5,30-34,52,9.03%
6,35-39,31,5.38%
7,40+,12,2.08%


In [34]:
## Age Demographics - Create Data Frame to seperate users by age into bins

age_group_df = purchases_df.groupby(['Age Group']).agg({'Purchase ID':'count','Price':['mean','sum']})

age_group_df.columns = age_group_df.columns.map('_'.join)
age_group_df = age_group_df.rename(columns={'Purchase ID_count':'Total Purchases',
                                            'Price_mean':'Average Price','Price_sum':'Total Sales'})

age_group_df = age_group_df.reset_index(drop=False)

average_per_age = []
for i in range(0,8):
    x = age_group_df.iloc[i,3]/age_total[i]
    average_per_age.append(x)

age_group_df['Average Price Per Person'] = average_per_age

age_group_df['Average Price'] = age_group_df['Average Price'].map('${:,.2f}'.format)
age_group_df['Total Sales'] = age_group_df['Total Sales'].map('${:,.2f}'.format)
age_group_df['Average Price Per Person'] = age_group_df['Average Price Per Person'].map('${:,.2f}'.format)


age_group_df

,Age Group,Total Purchases,Average Price,Total Sales,Average Price Per Person
0,<10,23,$3.35,$77.13,$4.54
1,10-14,28,$2.96,$82.78,$3.76
2,15-19,136,$3.04,$412.89,$3.86
3,20-24,365,$3.05,"$1,114.06",$4.32
4,25-29,101,$2.90,$293.00,$3.81
5,30-34,73,$2.93,$214.00,$4.12
6,35-39,41,$3.60,$147.67,$4.76
7,40+,13,$2.94,$38.24,$3.19


In [35]:
## Identify the the top 5 spenders in the game by total purchase value, then list (in a table):

new_group = purchases_df.groupby(['SN']).agg({'Purchase ID':['count'],
                                                    'Price':['sum']})
new_group.columns = new_group.columns.map('_'.join)

sorted_group = new_group.sort_values(['Price_sum'],ascending=False).head()

mylist = []
for i in range(0,5):
    price_sum = sorted_group['Price_sum'][i]
    purchase_count = sorted_group['Purchase ID_count'][i]
    average_price2 = price_sum/purchase_count
    mylist.append(average_price2)


sorted_group['Average purchase Price'] = mylist
sorted_group = sorted_group.rename(columns={'Purchase ID_count':'Purchases',
                                           'Price_sum':'Total Price',
                                            'Average purchase Price':'Average Price'})

sorted_group['Total Price'] = sorted_group['Total Price'].map('${:,.2f}'.format)
sorted_group['Average Price'] = sorted_group['Average Price'].map('${:,.2f}'.format)
sorted_group = sorted_group.reset_index(drop=False)

sorted_group


,SN,Purchases,Total Price,Average Price
0,Lisosia93,5,$18.96,$3.79
1,Idastidru52,4,$15.45,$3.86
2,Chamjask73,3,$13.83,$4.61
3,Iral74,4,$13.62,$3.40
4,Iskadarya95,3,$13.10,$4.37


In [36]:
## Identify the 5 most popular items by purchase count, then list (in a table):

items_df = purchases_df[['Item Name','Item ID','Price','SN']]
group_items_df = items_df.groupby(['Item Name','Item ID']).agg({'Price':['max','sum'],
                                                                'SN':'count'})
group_items_df.columns = group_items_df.columns.map('_'.join)
group_items_df = group_items_df.rename(columns={'Price_max':'Price',
                                               'Price_sum':'Total Sales',
                                               'SN_count':'Total Sold'})

group_items_df = group_items_df.reset_index(drop=False)
sorted_items_df = group_items_df.sort_values(['Total Sold'],ascending=False)

final_items_df = sorted_items_df.reset_index(drop=True).head()
final_items_df['Price'] = final_items_df['Price'].map('${:,.2f}'.format)
final_items_df['Total Sales'] = final_items_df['Total Sales'].map('${:,.2f}'.format)

final_items_df


,Item Name,Item ID,Price,Total Sales,Total Sold
0,Final Critic,92,$4.88,$59.99,13
1,"Oathbreaker, Last Hope of the Breaking Storm",178,$4.23,$50.76,12
2,Persuasion,132,$3.33,$28.99,9
3,Nirvana,82,$4.90,$44.10,9
4,"Extraction, Quickblade Of Trembling Hands",108,$3.53,$31.77,9


In [37]:
## Identify the 5 most profitable items by total purchase value, then list (in a table):

total_value_df = group_items_df.sort_values(['Total Sales'],ascending=False)
final_total_value_df = total_value_df.reset_index(drop=True).head()

final_total_value_df['Price'] = final_total_value_df['Price'].map('${:,.2f}'.format)
final_total_value_df['Total Sales'] = final_total_value_df['Total Sales'].map('${:,.2f}'.format)


final_total_value_df


,Item Name,Item ID,Price,Total Sales,Total Sold
0,Final Critic,92,$4.88,$59.99,13
1,"Oathbreaker, Last Hope of the Breaking Storm",178,$4.23,$50.76,12
2,Nirvana,82,$4.90,$44.10,9
3,Fiery Glass Crusader,145,$4.58,$41.22,9
4,Singed Scalpel,103,$4.35,$34.80,8
